In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import optimus as Optimus

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
C:\Users\argenisleon\Anaconda3\lib\site-packages\statsmodels\base\wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  defaults=argspec[3])

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [4]:
op = Optimus("dask")

In [5]:
df = op.load.csv("data/crime.csv")

In [9]:
%%time
df["INCIDENT_NUMBER"].value_counts().nlargest(5).compute()

Wall time: 4.16 s


I162030584    13
I152080623    11
I172013170    10
I182065208    10
I172096394    10
Name: INCIDENT_NUMBER, dtype: int64

In [11]:
df["INCIDENT_NUMBER"].nunique().compute()

282517

In [58]:
df.npartitions

1

In [71]:
df = df.repartition(npartitions=20)

In [72]:
df.npartitions

20

In [13]:
#count
len(df)

319073

In [73]:
%%time
df["INCIDENT_NUMBER"].value_counts().nlargest(5).compute()

Wall time: 8.07 s


I162030584    13
I152080623    11
I172013170    10
I182065208    10
I172096394    10
Name: INCIDENT_NUMBER, dtype: int64

In [9]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [16]:
df["INCIDENT_NUMBER"].unique().count().compute()

282517

In [6]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [20]:
%%time
agg_func=["min","max","mean"]
exprs = df.cols.create_exprs(["INCIDENT_NUMBER", "OFFENSE_CODE"], agg_func)
result = df.cols.exec_agg(exprs)
print(result)

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [15]:
import dask.dataframe as dd

In [26]:
%%time
dd.compute(df.min(), df["INCIDENT_NUMBER"].value_counts().nlargest(5), df["OFFENSE_CODE"].value_counts().nlargest(5))

Wall time: 7.35 s


(INCIDENT_NUMBER                                           142052550
 OFFENSE_CODE                                                    111
 OFFENSE_CODE_GROUP                               Aggravated Assault
 OFFENSE_DESCRIPTION    A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.
 REPORTING_AREA                                                     
 OCCURRED_ON_DATE                                2015-06-15 00:00:00
 YEAR                                                           2015
 MONTH                                                             1
 DAY_OF_WEEK                                                  Friday
 HOUR                                                              0
 Lat                                                              -1
 Long                                                       -71.1787
 Location                                 (-1.00000000, -1.00000000)
 dtype: object, I162030584    13
 I152080623    11
 I172013170    10
 I182065208    10
 I172096394    1

In [27]:
## https://distributed.dask.org/en/latest/web.html

NameError: name 'cluster' is not defined

In [56]:
# Another approach
# from dask.array import stats
# x = da.random.beta(1, 1, size=(1000,), chunks=10)
# k, s, m = [stats.kurtosis(x), stats.skew(x), stats.moment(x, 5)]
# dask.compute(k, s, m)

(1.7612340817172787, -0.064073498030693302, -0.00054523780628304799)

(1.7612340817172787, -0.0640734980306933, -0.000545237806283048)

In [16]:
data = [['tom', 10], ['nick', 15], ['juli', 0],['argenis', 10]]

# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Name', 'Age']) 
df.cols.count_by_dtypes("*")

NameError: name 'pd' is not defined

In [58]:
df

,Name,Age
0,tom,10
1,nick,15
2,juli,0
3,argenis,10


In [65]:
# df[df == 0].count(axis=0)
# Speedy https://stackoverflow.com/questions/35277075/python-pandas-counting-the-occurrences-of-a-specific-value
def zeros(series):
    return (series.values==0).sum()

zeros(df["Age"])

1

In [135]:
def percentile_agg(col_name, values):  
    return col_name.quantile(values).to_dict()
    

percentile_agg(df["Age"], [0.1, 0.5])

{0.1: 3.0000000000000004, 0.5: 10.0}

In [136]:
df.agg(
    {
#         # find the min, max, and sum of the duration column
#         'Name': [min, max, sum],
#          # find the number of network type entries
#         'Age': "count",
        # min, first, and number of unique dates per group
        'Age': [min, 'first', 'nunique', percentile_agg("Age",[0.1, 0.5])]
    }
).to_dict()

AttributeError: 'str' object has no attribute 'quantile'

In [37]:
print(df)

   Name Age
0   tom  10
1  nick  15
2  juli   0


In [42]:
from optimus.profiler.profiler import Profiler
p = Profiler()
p.columns_agg(df)

<optimus.dask.functions.functions.<locals>.Functions object at 0x000001F3BE39B400>


TypeError: count_uniques_agg() missing 1 required positional argument: 'col_name'

In [ ]:

#https://stackoverflow.com/questions/46080171/constructing-mode-and-corresponding-count-functions-using-custom-aggregation-fun

In [31]:


def chunk(s):
    """
    The function applied to the
    individual partition (map)
    """
    return s.apply(lambda x: list(set(x)))

def agg(s):
    """
    The function whic will aggrgate
    the result from all the partitions(reduce)
    """
    s = s._selected_obj
    return s.groupby(level=list(range(s.index.nlevels))).sum()

def finalize(s):
    """
    The optional functional that will be
    applied to the result of the agg_tu functions
    """
    return s.apply(lambda x: len(set(x)))

tunique = dd.Aggregation('tunique', chunk, agg, finalize)

In [42]:
df.groupby(['INCIDENT_NUMBER']).agg({'INCIDENT_NUMBER': tunique}).compute()

,INCIDENT_NUMBER
INCIDENT_NUMBER,
142052550,1
I010370257-00,1
I030217815-08,1
I050310906-00,1
I060168073-00,1
I080542626-00,1
I090317057-00,1
I090321958-00,1
I100033064-00,1


distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/10 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/11 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/12 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/13 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/14 remote=inproc://192.168.0.9/34804/1>


In [34]:
dd.compute(df.tunique())

TypeError: 'Aggregation' object is not callable

In [40]:
df.apply({'g0': tunique}, axis=1).compute()

`meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("'Aggregation' object is not callable", 'occurred at index 0')

Traceback:
---------
  File "C:\Users\argenisleon\Anaconda3\lib\site-packages\dask\dataframe\utils.py", line 137, in raise_on_meta_error
    yield
  File "C:\Users\argenisleon\Anaconda3\lib\site-packages\dask\dataframe\core.py", line 3601, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Users\argenisleon\Anaconda3\lib\site-packages\dask\utils.py", line 694, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "C:\Users\argenisleon\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py", line 6487, 